# Mock Photon Simulation - Usage

This Notebook serves the purpose to show how you can use the Mock Photon propagation. Let's start with the imports

In [ ]:
import sys

sys.path.append('../')

from olympus.event_generation.generators import get_generator
from olympus.configuration.generators import EventGeneratorConfiguration
from olympus.configuration.generators import GenerationConfiguration
from olympus.event_generation.medium import MediumEstimationVariant
from olympus.configuration.generators import UniformSpectrumConfiguration
from ananke.schemas.event import EventType
from olympus.configuration.photon_propagation import MockPhotonPropagatorConfiguration
from olympus.event_generation.photon_propagation.mock_photons import MockPhotonPropagator
from olympus.configuration.generators import DatasetConfiguration






### Use Mock Photon Simulation in Pipeline

You should not be obliged to actually call the Simulation itself. Rather than that you should pass it to the generation pipeline. For the optimal pipeline, we need a generator, the hit type we want to generate and a photon propagator, where we will use the mock propagator

In [ ]:


from ananke.configurations.presets.detector import single_line_configuration
from olympus.event_generation.generators import generate

photon_propagator_configuration = MockPhotonPropagatorConfiguration(
    resolution=18000,
    medium=MediumEstimationVariant.PONE_OPTIMISTIC
)

configuration = DatasetConfiguration(
    detector=single_line_configuration,
    generators=[
        GenerationConfiguration(
            generator=EventGeneratorConfiguration(
                type=EventType.CASCADE,
                spectrum=UniformSpectrumConfiguration(
                    log_minimal_energy=2.0,
                    log_maximal_energy=5.5
                ),
                source_propagator=photon_propagator_configuration
            ),
            number_of_samples=1
        )
    ],
    data_path="data/nb_mock_photon_usage"
)

collection = generate(configuration)

Let's see what we got!

In [ ]:
print('Number of Collection: ', len(collection.records))
records = collection.get_records()
records.df.head()

In [ ]:
print('Number of Sources: ', len(collection.sources))
sources = collection.get_sources()
sources.df.head()

In [ ]:
print('Number of Hits: ', len(collection.hits))
collection.hits.df.head()

## Use Mock Photon Propagator alone

If you want to call it yourself, you only need sources and the according events.

In [ ]:
from olympus.event_generation.photon_propagation.mock_photons import MockPhotonPropagator
from ananke.services.detector import DetectorBuilderService

builder = DetectorBuilderService()
det = builder.get(single_line_configuration)
propagator = MockPhotonPropagator(
    detector=det,
    configuration=photon_propagator_configuration
)
propagator.propagate(collection.get_records(), collection.get_sources())
